## Imports

In [1]:
import numpy as np
import scipy as sp

import tensorflow as tf

import heapq
import math

## Load Dataset

In [2]:
def load_rating_file_as_list(filename):
    rating_list = []
    
    with open(filename, "r") as f:
        line = f.readline()
        
        while line and line != "":
            arr = line.split("\t")
            user, item = int(arr[0]), int(arr[1])
            rating_list.append([user, item])
            line = f.readline()
    
    return rating_list

def load_negative_file(filename):
    negative_list = []
    
    with open(filename, "r") as f:
        line = f.readline()
        
        while line and line != "":
            arr = line.split("\t")
            negatives = []
            
            for x in arr[1:]:
                negatives.append(int(x))
            
            negative_list.append(negatives)
            
            line = f.readline()
    
    return negative_list

def load_rating_file_as_matrix(filename):
    num_users, num_items = 0, 0
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            num_users = max(num_users, u)
            num_items = max(num_items, i)
            line = f.readline()
    
    mat = sp.sparse.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            if (rating > 0):
                mat[user, item] = 1.0
            line = f.readline()    
    return mat

In [3]:
train = load_rating_file_as_matrix('./Data/ml-1m/ml-1m.train.rating')
test_ratings = load_rating_file_as_list('./Data/ml-1m/ml-1m.test.rating')
test_negatives = load_negative_file('./Data/ml-1m/ml-1m.test.negative')

In [4]:
num_users, num_items = train.shape
print('Loaded Data. # Users:', num_users, '# Items:', num_items, '# Train:', train.nnz, '# Test:', len(test_ratings))

Loaded Data. # Users: 6040 # Items: 3706 # Train: 994169 # Test: 6040


### Helper to Generate Negative Training Examples

In [5]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while train.get((u, j)):
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

## Create Model

In [6]:
from keras.models import Model
from keras.layers import Embedding, Input, Dense, Reshape, Multiply, Flatten, Lambda, Concatenate, Conv2D, MaxPool2D
from keras import initializers, regularizers
import sys

def get_OuterProductmodel(num_users, num_items, latent_dim):
    user_input = Input(shape = (1,), dtype = 'int32', name = 'user')
    item_input = Input(shape = (1,), dtype = 'int32', name = 'item')

    user_embedding = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
    item_embedding = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)

    user_latent = Flatten()(user_embedding(user_input))
    item_latent = Flatten()(item_embedding(item_input))

    latent_map = tf.linalg.matmul(tf.expand_dims(user_latent, -1), tf.expand_dims(item_latent, 1))

    x = tf.expand_dims(latent_map, -1)

    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = Flatten()(x)
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(x)

    return Model(inputs=[user_input, item_input], outputs=prediction)

In [7]:
from keras.optimizers import Adam

topK = 10

model = get_OuterProductmodel(num_users, num_items, 8)
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
item (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embed (Embedding)          (None, 1, 8)         48320       user[0][0]                       
__________________________________________________________________________________________________
item_embed (Embedding)          (None, 1, 8)         29648       item[0][0]                       
______________________________________________________________________________________________

## Define Evaluation Functions

In [8]:
def evaluateNDCG(ranked_list, target_item):
    for i in range(len(ranked_list)):
        if ranked_list[i] == target_item:
            return math.log(2) / math.log(i + 2)
  
    return 0

def hitRate(ranked_list, target_item):
    for rank in ranked_list:
        if target_item == rank:
            return 1
    return 0

# This method calculates all the evaluation metrics. Individual methods are called from here.
def evaluate(model, testPosRatings, testNegRatings, N):
    hits = []
    ndcgs = []
    for i in range(len(testPosRatings)):
        hit, ncdg = evaluate_one(model, testPosRatings[i], testNegRatings[i], N)
        hits.append(hit)
        ndcgs.append(ncdg)
        
    return np.array(hits).mean(), np.array(ndcgs).mean()

def evaluate_one(model, posRating, negRatings, N):
    user = posRating[0]
    movie = posRating[1]
    negRatings.append(movie)

    user_input = np.full(len(negRatings), user)

    predictions = model.predict([user_input, np.array(negRatings)], batch_size = 100)

  # associate item with predictions
    items = {}
    for i in range(len(predictions)):
        items[negRatings[i]] = predictions[i]
    negRatings.pop()

    rankedList = heapq.nlargest(N, items, items.get)
    ndcg = evaluateNDCG(rankedList, movie)
    hit = hitRate(rankedList, movie)

    return hit, ndcg

## Train and Save Best Model

In [9]:
NUM_EPOCHS = 10
best_hr = 0
best_ncdg = 0
best_epoch = -1
model_path = "OP_model.h5"

hit_rate, ncdg = evaluate(model, test_ratings, test_negatives, N = 10)
print('Initial Model', 'Hit Rate:', hit_rate, 'NCDG:', ncdg)

for epoch in range(1, NUM_EPOCHS + 1):
    user_input, item_input, labels = get_train_instances(train, num_negatives = 4)

    hist = model.fit([np.array(user_input), np.array(item_input)],
                      np.array(labels),
                      batch_size = 256, epochs = 1)

    hit_rate, ncdg = evaluate(model, test_ratings, test_negatives, N = 10)
    print('Epoch', epoch, 'Hit Rate:', hit_rate, 'NCDG:', ncdg)
    model.save(model_path)

    if hit_rate > best_hr:
        best_hr, best_ncdg, best_iter = hit_rate, ncdg, epoch
        model.save(model_path, overwrite=True)

print("Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ncdg))
print("The best Outer Product model is saved to %s" %(model_path))

Initial Model Hit Rate: 0.09437086092715231 NCDG: 0.04076982868985803
19418/19418 [==============================] - 233s 12ms/step - loss: 0.3799
Epoch 1 Hit Rate: 0.4768211920529801 NCDG: 0.25626444996711006
  158/19418 [..............................] - ETA: 3:51 - loss: 0.3399

KeyboardInterrupt: 